# 1. Preprocessing the input images
## 1.1 Loading Data

In [ ]:
!pip install --upgrade pip -q
!pip install --upgrade tensorflow --log LOGFILE
import tensorflow as tf
!pip  install openpyxl
tf.__version__

In [ ]:
import numpy as np 
import pandas as pd 
import cv2 
import matplotlib.pyplot as plt 
import os 
!pip install rarfile
import rarfile
import  PIL.Image as im
from scipy.ndimage import interpolation as inter
from tensorflow import keras
#import tensorflow  as tf 
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.losses import categorical_crossentropy

In [ ]:
images_path = "../input/dataset/data_nh_2021/dbnh_2021"
labels_path = "../input/datass/code_bdnh_2021.xlsx"
images_files = os.listdir(images_path)

In [ ]:
## Display some images
i=15
fig, axs = plt.subplots(2,2) 
for x in range(2):
    for y in range(2):
        img = cv2.imread(images_path +"/"+ images_files[i], cv2.IMREAD_UNCHANGED)
        i=i+1 
        axs[x,y].imshow(img) 

In [ ]:
### Build dataset
dataset = np.ndarray((5279,190,1061))
#max_x = 190
#max_y = 1061
for i in range(len(images_files)):
    data = cv2.imread(images_path +"/"+ images_files[i], cv2.IMREAD_UNCHANGED)
    x= data.shape[0]
    y= data.shape[1]
    imgg = np.pad(data,((int (np.ceil((190-x) /2)),int(np.ceil((190-x )/2))), (int(np.ceil((1061-y ))/2),int(np.ceil((1061-y)/2)))), 'constant')
    dataset[i,:,:]= imgg[:190, :1061]

In [ ]:
plt.imshow(dataset[0,:,:])

In [ ]:
### Split data
y = pd.read_excel('../input/yylabels/code_bdnh_2021.xlsx', header=None)
#X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.33, random_state=42)
X_train = dataset[:1500,:,:]
X_test =  dataset[4000:5000,:,:]
X_val= dataset[5000:,:,:]
y_train = y[0][:1500]
y_test = y[0][4000:5000]
y_val= y[0][5000:]


In [ ]:
## images shape 
shapes = []
for file in images_files:
      shapes.append(cv2.imread(images_path +"/"+ images_files[i], cv2.IMREAD_UNCHANGED).shape)

## 1.2 Preprocessing data
• **Filtrage** -- (*`Filtre local`* --> `NETTOYAGE` --> modifier la valeur d’un pixel blanc(1)en un pixel noir(0) si son voisinage 3X3 contient au moins cinq pixels noirs(0) et vis-versa.) \
• **Binarisation**  -- >` deja faite` \
• **Squelettisation** --> L’algorithme de `*Zhang & Suen*` a le mieux squelettis´e les mots de la base
A skeleton can be computed using only the two basic morphological operations: dilate and erode

IFN/ENIT.\
• **D´etection et redressement de la ligne de base**

### 1.2.1 Filtrage:

In [ ]:
def remove_noise(image):
    return cv2.medianBlur(image,5)
    
i=15
fig, axs = plt.subplots(2,2) 
for x in range(2):
    for y in range(2):
        img = cv2.imread(images_path +"/"+ images_files[i], cv2.IMREAD_UNCHANGED)
        i=i+1 
         # cv2.imshow('img',img) //  cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
        img =squel(img) 
        ## TO DO: specify the output folder for the processed images
        axs[x,y].imshow(img) 
        
outputDir = "/content/sample_data/prepData/"
for file in images_files:
    img = cv2.imread(images_path +"/"+ file, cv2.IMREAD_UNCHANGED)
    img =remove_noise(img) 
    cv2.imwrite(outputDir+file, img)

### 1.2.2 Squelettisation

In [ ]:
def squel(img):
    img = cv2.imread(images_path +"/"+ images_files[3],0)
    kernel = np.ones((3,3),np.uint8)
    erosion = cv2.erode(img,kernel,iterations = 1)
    return img

for x in range(2):
    for y in range(2):
        img = cv2.imread(images_path +"/"+ images_files[i], cv2.IMREAD_UNCHANGED)
        i=i+1 
        img =squel(img) 
        axs[x,y].imshow(img) 

### 1.2.3 D´etection et redressement de la ligne de base

In [ ]:
h,w = img.shape
lignes = np.zeros((h),np.uint16)
cols = np.zeros((w),np.uint16)

# nombre de pix noirs
for i in range(h):
    for j in range(w):
        if img[i,j] ==0:
            lignes[i]+=1
            cols[j]+=1

imgLignes = np.zeros(img.shape, np.uint8)
imgLignes = np.zeros(img.shape, np.uint8)
imgCols = np.zeros(img.shape, np.uint8)
imgLignes [:,:]= 255
imgCols [:,:]= 255

## creation du histogramme 
for i in range(h):
    for j in range(lignes[i]):
        imgLignes[i,j]=0
for j in range(w):
    for i in range(cols[j]):
        imgCols[i,j]=0

In [ ]:
img = im.open(images_path +"/"+ images_files[10])
# convert to binary
wd, ht = img.size
pix = np.array(img.convert('1').getdata(), np.uint8)
bin_img = img

#bin_img = 1 - (pix.reshape((ht, wd)) / 255.0)

def find_score(arr, angle):
    data = inter.rotate(arr, angle, reshape=False, order=0)
    hist = np.sum(data, axis=1)
    score = np.sum((hist[1:] - hist[:-1]) ** 2)
    return hist, score

delta = 1
limit = 5
angles = np.arange(-limit, limit+delta, delta)
scores = []
for angle in angles:
    hist, score = find_score(bin_img, angle)
    scores.append(score)

best_score = max(scores)
best_angle = angles[scores.index(best_score)]
print('Best angle: {}'.format(best_angle))
# correct baseline 
data = inter.rotate(bin_img, best_angle, reshape=False, order=0)
img = im.fromarray((255 * data).astype("uint8")).convert("RGB")
##img.save('skew_corrected.png')

def redresse(img):
    delta = 1
    limit = 5
    angles = np.arange(-limit, limit+delta, delta)
    scores = []
    for angle in angles:
        hist, score = find_score(bin_img, angle)
        scores.append(score)

    best_score = max(scores)
    best_angle = angles[scores.index(best_score)]
    # correct baseline 
    data = inter.rotate(bin_img, best_angle, reshape=False, order=0)
    img = im.fromarray((255 * data).astype("uint8")).convert("RGB")
    return img

# ADD LOOP AND SAVE NEW IMAGES
for file in images_files:
    img = cv2.imread(outputDir +"/"+ file, cv2.IMREAD_UNCHANGED)
    img =redresse(img) 
    img.save(outputDir+file)
  

In [ ]:
plt.imshow(img)

# 2. Building model 

In [ ]:
class Model(Sequential):
    def __init__(self):
        super().__init__()
        # alexnet inspired
        self.add(keras.layers.Rescaling(1./255, input_shape=(190, 1061,1)))
        self.add(layers.Conv2D(16, 3, padding='same', activation='relu',  )),
        self.add(Dropout(0.4))
        self.add(keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))),
        # vgg19 inspired
        self.add(Conv2D(32, kernel_size=(3,3), padding= 'same',activation= 'relu'))
        self.add(Dropout(0.4))
        self.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))
        self.add(Dropout(0.4))
        self.add(Conv2D(64, kernel_size=(3,3), padding= 'same',activation= 'relu'))
        self.add(Conv2D(128, kernel_size=(3,3), padding= 'same',activation= 'relu'))
        self.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))
        self.add(Flatten())
        self.add(Dense(120, activation= 'relu'))
        self.add(Dropout(0.5))
        self.add(Dense(22, activation= 'softmax'))
        self.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model = Model()
model.summary()

In [ ]:
## Training
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history  = model.fit(X_train3,y_train3,epochs =30, validation_data=(X_val,y_val), callbacks=[es_callback])
model.save("./Model")

In [ ]:
## Learning curves
epochs = 8 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(6, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
pred = model.predict(X_test)
## Get the classe
yPredFinal = [np.argmax(x) for x in pred] 

In [ ]:
from sklearn.metrics import f1_score
f1_score (y_test,yPredFinal, average=None)

In [ ]:
## prepapre second batch of training data
from sklearn.metrics import accuracy_score
accuracy_score(y_test,yPredFinal)

# 3. Augment Data

In [ ]:
def augment_using_opsParms1(images, labels):
    images = tf.image.random_flip_left_right(images)
                                  zoom_range=(0.7, 1.0)    
    size = images.shape[0]
    augImages = np.ndarray((size,190,1061))
    for i in range(size) :
        augImages[i,:,:] =tfa.image.rotate(images[i,:,:], tf.constant(np.pi/9))
    return (augImages, labels)

In [ ]:
def augment_using_opsParms2(images, labels):
    images = tf.image.random_flip_left_right(images)
    size = images.shape[0]
    augImages = np.ndarray((size,190,1061))
    for i in range(size) :
        augImages[i,:,:] =tfa.image.rotate(images[i,:,:], tf.constant(np.pi/20))
        
    return (augImages, labels)

In [ ]:
augImgs2, augLabels2 = augment_using_opsParms2(dataset[:1125,:,:], y[0][:1125])
np.save("./augImgs2Final.npy", augImgs1)
np.save("./augLabels2Final.npy", augLabels1)